In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [2]:
df = pd.read_csv('./sample_data_feb_2018.csv')
df1 = pd.read_csv('./sample_data_mar_2018.csv')

df = pd.concat([df, df1])

In [3]:
df.head()

,departure_date,airline_code,flight_number,source_airport,destination_airport,flight_status,departure_scheduled_time,departure_actual_time,arrival_scheduled_time,arrival_actual_time
0,2018-02-01,EK,5010,LHR,DXB,LANDED,1.517486e+09,1517486460,1.517512e+09,1.517511e+09
1,2018-02-01,IB,4649,LHR,BWI,LANDED,1.517498e+09,1517498940,1.517528e+09,1.517530e+09
2,2018-02-01,NZ,4292,LHR,AUH,LANDED,1.517476e+09,1517475840,1.517502e+09,1.517500e+09
3,2018-02-01,KE,503,LHR,CDG,LANDED,1.517516e+09,1517516820,1.517521e+09,1.517521e+09
4,2018-02-01,JJ,2861,LHR,GVA,LANDED,1.517505e+09,1517504460,1.517511e+09,1.517510e+09


In [4]:
df.dtypes

departure_date               object
airline_code                 object
flight_number                 int64
source_airport               object
destination_airport          object
flight_status                object
departure_scheduled_time    float64
departure_actual_time         int64
arrival_scheduled_time      float64
arrival_actual_time         float64
dtype: object

In [5]:
# First convert all the dates to date time

df['departure_date'] = pd.to_datetime(df['departure_date'])
df['departure_scheduled_time'] = pd.to_datetime(df['departure_scheduled_time'], unit='s')
df['departure_actual_time'] = pd.to_datetime(df['departure_actual_time'], unit='s')
df['arrival_scheduled_time'] = pd.to_datetime(df['arrival_scheduled_time'], unit='s')
df['arrival_actual_time'] = pd.to_datetime(df['arrival_actual_time'], unit='s')

In [6]:
df.dtypes

departure_date              datetime64[ns]
airline_code                        object
flight_number                        int64
source_airport                      object
destination_airport                 object
flight_status                       object
departure_scheduled_time    datetime64[ns]
departure_actual_time       datetime64[ns]
arrival_scheduled_time      datetime64[ns]
arrival_actual_time         datetime64[ns]
dtype: object

In [10]:
# Let's find any missing values
df.isnull().sum()


departure_date                 0
airline_code                   0
flight_number                  0
source_airport                 0
destination_airport            0
flight_status                  0
departure_scheduled_time     794
departure_actual_time          0
arrival_scheduled_time       768
arrival_actual_time         2086
dtype: int64

In [14]:
# Total missing values
print(f'total missing values {df.isnull().sum().sum()}')

total missing values 3648


In [17]:
# Since only a small poertion of the data is missing we can remove them
missing_data_fraction = (df.isnull().sum().sum())/df.shape[0]
print('Fraction of missing data: ', missing_data_fraction*100, "%")

Fraction of missing data:  3.4989113858489755 %


In [18]:
df.dropna(inplace=True)


In [26]:
unique_flights = df.groupby('airline_code')['flight_number'].nunique()
print(unique_flights)

airline_code
4U     12
9W     17
A3     12
AA    339
AC     76
AD      1
AF     16
AH      1
AI     13
AM      6
AT      2
AV      8
AY     91
AZ      6
BA    484
BE      8
BG      1
BI      1
BM      3
BR      1
BT      1
CA      3
CI      1
CO      1
CX     41
CZ      8
DL     47
DT      1
EI     84
EK     11
     ... 
PR      1
QF     27
QR     28
RJ     12
RO      1
RU      1
S4      1
S7      5
SA     16
SK     28
SN     10
SQ     36
SU      9
SV      7
TG      4
TK      7
TP     20
TU      1
UA     80
UL      9
UU      1
VA      7
VN      3
VS     42
VY      3
WY      2
X3      3
XY      3
ZI      1
ZT      2
Name: flight_number, Length: 102, dtype: int64


In [27]:
df[(df['flight_status'] != "LANDED")]

,departure_date,airline_code,flight_number,source_airport,destination_airport,flight_status,departure_scheduled_time,departure_actual_time,arrival_scheduled_time,arrival_actual_time


In [28]:
#Landed feature is useless.
df.drop(['flight_status'], axis=1, inplace=True)

In [29]:
#Make dates categorical

df['departure_date_day_of_week']=df.departure_date.dt.dayofweek
df['departure_date_day_of_month']=df.departure_date.dt.day
df['departure_date_month_of_year']=df.departure_date.dt.month
df['departure_date_year']=df.departure_date.dt.year

df['departure_scheduled_time_day_of_week']=df.departure_scheduled_time.dt.dayofweek
df['departure_scheduled_time_day']=df.departure_scheduled_time.dt.day
df['departure_scheduled_time_month_of_year']=df.departure_scheduled_time.dt.month
df['departure_scheduled_time_year']=df.departure_scheduled_time.dt.year

df['departure_actual_time_day_of_week']=df.departure_actual_time.dt.dayofweek
df['departure_actual_time_day']=df.departure_actual_time.dt.day
df['departure_actual_time_month_of_year']=df.departure_actual_time.dt.month
df['departure_actual_time_year']=df.departure_actual_time.dt.year

df['arrival_scheduled_time_day_of_week']=df.arrival_scheduled_time.dt.dayofweek
df['arrival_scheduled_time_day']=df.arrival_scheduled_time.dt.day
df['arrival_scheduled_time_month_of_year']=df.arrival_scheduled_time.dt.month
df['arrival_scheduled_time_year']=df.arrival_scheduled_time.dt.year

df['arrival_actual_time_day_of_week']=df.arrival_actual_time.dt.dayofweek
df['arrival_actual_time_day']=df.arrival_actual_time.dt.day
df['arrival_actual_time_month_of_year']=df.arrival_actual_time.dt.month
df['arrival_actual_time_year']=df.arrival_actual_time.dt.year

In [30]:
# Dealyed target
df['arrival_delta_minutes']=(df.arrival_actual_time-df.arrival_scheduled_time).astype('timedelta64[m]')

In [31]:
# Convert airline_code, flight_code and destination_airport to categorical
# airline_code flight_number source_airport destination_airport
df.airline_code = df.airline_code.astype('category')
df.flight_number = df.flight_number.astype('category')
df.source_airport = df.source_airport.astype('category')
df.destination_airport = df.destination_airport.astype('category')


In [33]:
print(f'Number of flight {df.flight_number.nunique()}')

Number of flight 1969


In [34]:
X = df[['airline_code', 'destination_airport', 'flight_number',
        'departure_date_day_of_week',
        'departure_date_day_of_month', 'departure_date_month_of_year',
        'departure_date_year','departure_scheduled_time_day_of_week',
        'departure_scheduled_time_day', 'departure_scheduled_time_month_of_year',
        'departure_scheduled_time_year', 'departure_actual_time_day_of_week',
        'departure_actual_time_day', 'departure_actual_time_month_of_year',
        'departure_actual_time_year', 'arrival_scheduled_time_day_of_week',
        'arrival_scheduled_time_day', 'arrival_scheduled_time_month_of_year',
        'arrival_scheduled_time_year', 'arrival_actual_time_day_of_week',
        'arrival_actual_time_day', 'arrival_actual_time_month_of_year',
        'arrival_actual_time_year','arrival_delta_minutes'
       ]]

In [36]:
df['delayed'] = df['arrival_delta_minutes'] > 30
df.delayed = df.delayed.astype('int')

Y = df['delayed']


In [38]:
Y.head()

0    0
1    1
2    0
3    0
4    0
Name: delayed, dtype: int64

In [39]:
# One hot encode all the categorical features.
one_hot = pd.get_dummies(X, columns=["airline_code", "destination_airport", "flight_number"], prefix=["airline_code", "destination_airport", "flight_number"])
X = pd.concat([one_hot,X], axis=1)
X.drop(['airline_code','destination_airport','flight_number'], inplace=True, axis=1)

In [54]:
Y.shape

(100637,)

In [56]:
print(f'Number of True: {Y.sum()}')
print(f'Number of False: {Y.shape[0]-Y.sum()}')

Number of True: 11855
Number of False: 88782


In [40]:
X.head()

,departure_date_day_of_week,departure_date_day_of_month,departure_date_month_of_year,departure_date_year,departure_scheduled_time_day_of_week,departure_scheduled_time_day,departure_scheduled_time_month_of_year,departure_scheduled_time_year,departure_actual_time_day_of_week,departure_actual_time_day,...,departure_actual_time_year,arrival_scheduled_time_day_of_week,arrival_scheduled_time_day,arrival_scheduled_time_month_of_year,arrival_scheduled_time_year,arrival_actual_time_day_of_week,arrival_actual_time_day,arrival_actual_time_month_of_year,arrival_actual_time_year,arrival_delta_minutes
0,3,1,2,2018,3,1,2,2018,3,1,...,2018,3,1,2,2018,3,1,2,2018,-17.0
1,3,1,2,2018,3,1,2,2018,3,1,...,2018,3,1,2,2018,4,2,2,2018,32.0
2,3,1,2,2018,3,1,2,2018,3,1,...,2018,3,1,2,2018,3,1,2,2018,-37.0
3,3,1,2,2018,3,1,2,2018,3,1,...,2018,3,1,2,2018,3,1,2,2018,4.0
4,3,1,2,2018,3,1,2,2018,3,1,...,2018,3,1,2,2018,3,1,2,2018,-13.0


In [42]:
# Random Forest ftw!
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

clf = RandomForestClassifier()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5, random_state=42)
clf.fit(X_train, y_train)
print(f' Accuracy: {clf.score(X_test, y_test)}')

 Accuracy: 0.9999006339553648


We can cleary see that the data seems to be overfitting. It might be because of the sparsity in the data or maybe because the data is sparsely distributed. Only 11855 is True 88782 is False(Around 7.5  times more than True).
The accuracy can be measured better if we get a new seperate test set.